This script calibrates BEACO2N carbon monoxide data using QuantAQ sensors as reference. 

In [41]:
# TODO: Merge these imports into the cells where they are used. 
import os
from glob import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import typing as t
# %pip install pyaqsapi
# %pip install certifi
# %pip install requests
import pyaqsapi as aqs
from datetime import date
from functools import reduce

In [47]:
aqs.aqs_credentials(username="david_mike_woodilla@brown.edu", key="ochrecrane43")
# RIDEM @ CCRI codes and parameters:
#   stateFIPS = '44'
#   countycode = '007'
#   site = '0022'
# Parameter no. for carbon monoxide: '42101'
# display(aqs.bysite.sampledata(
#     parameter='42101', 
#     bdate=date(2025,1,1), 
#     edate=date(2025,1,31), 
#     stateFIPS='44',
#     countycode='007',
#     sitenum='0040'
# ))
# bdate = date(2023, 1, 1)
# edate = date(2023,12,31)
# display(aqs.bystate.monitors(parameter='42101,42102', bdate=bdate, edate=edate, stateFIPS='44').T)

# display(aqs.bysite.sampledata(
#     parameter='42101', 
#     bdate=bdate, 
#     edate=edate, 
#     stateFIPS='44',
#     countycode='007',
#     sitenum='0040'
# ))
byear = 2022
eyear = 2025
aqs_df = [pd.DataFrame for _ in range(eyear-byear)]
for i, year in enumerate(range(byear, eyear)):
    print(i)
    aqs_df[i] = aqs.bysite.sampledata(
        parameter='42101', 
        bdate=date(year, 1, 1), 
        edate=date(year,12,31), 
        stateFIPS='44',
        countycode='007',
        sitenum='0040'
    )

# if type(aqs_df) != pd.DataFrame: raise TypeError("aqs_df is of type AQSAPI_V2, should be of type pd.DataFrame")
aqs_df = pd.concat(aqs_df, join='outer', sort=False)
display(aqs_df)


0
1
2


,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,date_local,...,detection_limit,uncertainty,qualifier,method_type,method,method_code,state,county,date_of_last_change,cbsa_code
0,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2022-11-11,...,0.04,None,BF - Precision/Zero/Span.,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2023-03-28,39300
1,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2022-11-11,...,0.04,None,BF - Precision/Zero/Span.,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2023-03-28,39300
2,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2022-11-11,...,0.04,None,BF - Precision/Zero/Span.,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2023-03-28,39300
3,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2022-11-15,...,0.04,None,BF - Precision/Zero/Span.,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2023-03-28,39300
4,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2022-11-15,...,0.04,None,BF - Precision/Zero/Span.,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2023-03-28,39300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2024-12-31,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2025-03-12,39300
8780,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2024-12-31,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2025-03-12,39300
8781,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2024-12-31,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2025-03-12,39300
8782,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2024-12-31,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2025-03-12,39300


Clean and pre-process raw data.

In [51]:
display(aqs_df.dropna(subset="sample_measurement"))

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,date_local,...,detection_limit,uncertainty,qualifier,method_type,method,method_code,state,county,date_of_last_change,cbsa_code
169,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2022-11-11,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2023-03-28,39300
170,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2022-11-11,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2023-03-28,39300
171,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2022-11-11,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2023-03-28,39300
172,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2022-11-11,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2023-03-28,39300
173,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2022-11-11,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2023-03-28,39300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2024-12-31,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2025-03-12,39300
8780,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2024-12-31,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2025-03-12,39300
8781,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2024-12-31,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2025-03-12,39300
8782,44,007,0040,42101,1,41.771987,-71.427881,WGS84,Carbon monoxide,2024-12-31,...,0.04,None,None,FRM,INSTRUMENTAL - Gas Filter Correlation Thermo E...,554,Rhode Island,Providence,2025-03-12,39300


In [ ]:
# Parse the measurement and reference dataframe lists from csv files in project folder.
measurement_files = glob("./BEACO2N_measurements/*.csv")
measurement_df = {os.path.splitext(os.path.basename(f))[0] : pd.read_csv(f) for f in measurement_files}
reference_files = glob("./reference_measurements/*.csv")
reference_df = {os.path.splitext(os.path.basename(f))[0] : pd.read_csv(f) for f in reference_files}

# Clean measurement and reference data.

def clean_measurement(df: pd.DataFrame) -> pd.DataFrame :
    # Store time in Pandas datetime format.
    df = df.rename(columns={"datetime":"timestamp", "co2_raw":"co2"})
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True).dt.round("h")

    # Drop redundant time columns
    df = df.drop(columns=[col for col in ["local_timestamp", "epoch", "node_file_id", "node_id"] if col in df.columns])
    
    # For all columns suffixed by "_wrk_aux", convert from Volts to milliVolts (*1000) and remove suffix
    wrk_aux_cols = df.filter(regex=r"_wrk_aux$").columns
    df[wrk_aux_cols] *= 1000
    df.rename(columns= {col : col.replace("_wrk_aux", "") for col in wrk_aux_cols}, inplace=True)
    
    # Drop all datapoints with incomplete data (e.g. missing co measurement)
    df = df.dropna()
    return df

# Clean each site's dataframe
measurement_df = {site: clean_measurement(df) for site, df in measurement_df.items()}

# Clean data for reference (QuantAQ) analogously
def clean_reference(df: pd.DataFrame) -> pd.DataFrame :
    df = df.drop(columns=[col for col in ["period_start", "period_end", "period_end_utc", "sn"] if col in df.columns])
    df = df.rename(columns={"period_start_utc": "timestamp", "pm25": "pm2_5"})
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
    df = df.dropna()
    return df

reference_df = {key: clean_reference(df) for key, df in reference_df.items()}

In [ ]:
# Remove CO outliers from every site's data, including reference. 
def rm_outliers(df : pd.DataFrame) -> pd.DataFrame :
    co_zscore = abs(df['co']-df['co'].mean())/df['co'].std()
    return df[co_zscore < 3]

measurement_df = {site: rm_outliers(df) for site, df in measurement_df.items()}
reference_df = {key: rm_outliers(df) for key, df in reference_df.items()}

### Distributed Calibration:
Find time intervals when RSD(co) < 0.10 for all reference sensors in the network:

In [ ]:
from functools import reduce
rsd_dist = []

# Add each site's timestamp and co data to a list of co tables with site column headers.
[rsd_dist.append(df[["timestamp","co"]].rename(columns={'co':site})) for site, df in reference_df.items()]
# Merge the co tables into one table. 
rsd_dist = reduce(lambda table, to_merge: pd.merge(table, to_merge, on="timestamp", how="inner"), rsd_dist)

def rsd(row:pd.Series) -> float:
    ''' Helper function to calculate residual standard deviation of a dataframe row. '''
    vals = row[1:].values.astype(float)
    mean = np.mean(vals)
    sd = np.std(vals)
    return float(sd/mean) if mean != 0 else np.nan

rsd_dist["rsd"]=rsd_dist.apply(rsd, axis=1)
# display(rsd_dist)
timestamps_rsd_lt_10pc = rsd_dist[rsd_dist["rsd"]<.10]["timestamp"]
timestamps_rsd_gt_10pc = rsd_dist[rsd_dist["rsd"]>=.10]["timestamp"]
# print(len(timestamps_rsd_lt_10pc))
# print(len(timestamps_rsd_gt_10pc))

# Filter datasets to include only data contained by 
# intersection(timestamps_rsd_lt_10pc, measurement[site]["timestamp"], reference["timestamp"])

ref_times_rsd_lt_10pc = timestamps_rsd_lt_10pc
'''A list of timestamps contained in all of: timestamps_rsd_lt_10pc, dpw, pema, pha'''
for ref_site in reference_df: ref_times_rsd_lt_10pc = ref_times_rsd_lt_10pc[ref_times_rsd_lt_10pc.isin(reference_df[ref_site]["timestamp"])]

meas_times_rsd_lt_10pc = {}
'''A map from measurement sites to list of timestamps containing ⋂(ref_times_rsd_lt_10pc, meas_site["timestamps"])'''
for meas_site in measurement_df.keys(): meas_times_rsd_lt_10pc[meas_site] = ref_times_rsd_lt_10pc[ref_times_rsd_lt_10pc.isin(measurement_df[meas_site]["timestamp"])]

ref_times_rsd_gt_10pc = timestamps_rsd_gt_10pc
'''A list of timestamps contained in all of: timestamps_rsd_gt_10pc, dpw, pema, pha'''
for ref_site in reference_df: ref_times_rsd_gt_10pc = ref_times_rsd_gt_10pc[ref_times_rsd_gt_10pc.isin(reference_df[ref_site]["timestamp"])]

meas_times_rsd_gt_10pc = {}
'''A map from measurement sites to list of timestamps containing ⋂(ref_times_rsd_gt_10pc, meas_site["timestamps"])'''
for meas_site in measurement_df.keys(): meas_times_rsd_gt_10pc[meas_site] = ref_times_rsd_gt_10pc[ref_times_rsd_gt_10pc.isin(measurement_df[meas_site]["timestamp"])]



Display residual graphs and statistics for each site model. 

In [ ]:
import matplotlib.pyplot as plt
from numpy import sqrt
from sklearn.metrics import mean_absolute_error, r2_score

def plot_zones(
        zones: t.Dict[str, t.List[str]],
        models: t.Dict[str, t.Dict[str, LinearRegression]], 
        y_test: t.Dict[str, t.Dict[str, pd.Series]], 
        y_test_pred: t.Dict[str, t.Dict[str, pd.Series]]) \
    -> None :

    num_refsites = len(zones)
    max_num_meassites = max(len(sites) for sites in zones.values())
    fig, axes = plt.subplots(num_refsites, max_num_meassites, figsize=(4*max_num_meassites, 4*num_refsites), squeeze=False)
    statistics = []

    for row, (ref_site, meas_site_list) in enumerate(zones.items()):
        for col, meas_site in enumerate(meas_site_list):
            y_true = y_test[ref_site][meas_site] # pd.Series
            # print(type(y_true))
            y_pred = y_test_pred[ref_site][meas_site] # np.ndarray
            # print(type(y_pred))
            if len(y_true)==0 or len(y_pred)==0:
                axes[row][col].set_visible(False); continue
            if len(y_true) != len(y_pred): raise ValueError("Length discrepancy")
            if np.size(y_true) != np.size(y_pred): raise ValueError("Size discrepancy")
            # print(np.size(y_true))
            # print(np.size(y_pred))
            # display(y_true)
            # display(y_pred)
            residual = y_true - y_pred
            axes[row, col].scatter(y_true, residual, alpha=0.5)
            axes[row, col].axhline(0, color='red', linestyle='--')
            axes[row, col].set_title(f"{meas_site} vs {ref_site}\nR^2={r2_score(y_true, y_pred):.3f}, MAE={mean_absolute_error(y_true, y_pred):.2f}")
            axes[row, col].set_xlabel("Reference CO (mV)")
            axes[row, col].set_ylabel("Residual (True - Pred) (mV)")
            for unused_col in range(len(meas_site_list), max_num_meassites): axes[row, unused_col].set_visible(False)

            model = models[ref_site][meas_site]
            statistics.append({
                "Reference" : ref_site,
                "Measurement" : meas_site,
                "Determination R^2" : r2_score(y_true, y_pred),
                "Correlation r" : np.corrcoef(y_true, y_pred)[0,1], 
                "RMSE" : sqrt(np.mean((y_true-y_pred)**2)), 
                "MAE" : mean_absolute_error(y_true, y_pred), 
                "Intercept" : model.intercept_,
                "co_coef" : model.coef_[0],
                "temp_coef" : model.coef_[1],
                "rh_coef" : model.coef_[2], 
                'n' : len(y_true) 
            })
    
    statistics.append({
        "Reference" : 'Mean', 
        'Measurement': '', 
        'Determination R^2' : np.mean([val.get("Determination R^2") for val in statistics]), 
        'Correlation r' : np.mean([val.get("Correlation r") for val in statistics]), 
        'RMSE' : np.mean([val.get('RMSE') for val in statistics]), 
        'MAE' : np.mean([val.get('MAE') for val in statistics]), 
        'Intercept' : np.mean([val.get('Intercept') for val in statistics]), 
        'co_coef' : np.mean([val.get('co_coef') for val in statistics]), 
        'temp_coef' : np.mean([val.get('temp_coef') for val in statistics]), 
        'rh_coef' : np.mean([val.get('rh_coef') for val in statistics]), 
        'n' : np.mean([val.get('n') for val in statistics]), 
    })

    plt.tight_layout(); plt.show()
    display(pd.DataFrame(statistics).round(4))

Assign each measurement (BEACO2N) sensor to a reference sensor and train a calibration model (for each measurement sensor) using timestamps with RSC(co)<0.10 within the reference network.

Note: Measurement nodes are assigned to the nearest reference node according to calculations done in QGIS with Grace's BPP network map. May be worth confirming this using ArcGIS at some point. (Perhaps RIDEM data can eventually be used to improve spacial accuracy... some nodes are >2mi from reference.)

| Reference | Measurement locations |
|------------|-------------------------|
| dpw | reservoir, medschool, dpw, ccri, southprovlib, prek, gym, cfs, myron|
| pema | ecubed, rochambeau, smithhill, martialarts, blackstone, rocklib, provcollege, pema|
| pha | silverlake, carnevale, zuccolo, wecc, unitedway, pha, mtpleasant, ricollege|


Fit per-site regression models to the data where RSC(co)<0.10:

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

zones = {
    "dpw" : ["reservoir", "medschool", "dpw", "ccri", "southprovlib", "prek", "gym", "cfs", "myron"],
    "pema" : ["ecubed", "rochambeau", "smithhill", "martialarts", "blackstone", "rocklib", "provcollege", "pema"],
    "pha" : ["silverlake", "carnevale", "zuccolo", "wecc", "unitedway", "pha", "mtpleasant", "ricollege"]
}

dist_models = {ref_site : {meas_site : LinearRegression() for meas_site in zones[ref_site]} for ref_site in zones.keys()}
X_train = {ref_site : {meas_site : pd.DataFrame() for meas_site in zones[ref_site]} for ref_site in zones.keys()}
y_train = {ref_site : {meas_site : pd.Series(dtype=float) for meas_site in zones[ref_site]} for ref_site in zones.keys()}

X_test_lt_10pc_rsd = {ref_site : {meas_site : pd.DataFrame() for meas_site in zones[ref_site]} for ref_site in zones.keys()}
y_test_lt_10pc_rsd = {ref_site : {meas_site : pd.Series(dtype=float) for meas_site in zones[ref_site]} for ref_site in zones.keys()}
y_test_pred_lt_10pc_rsd = {ref_site : {meas_site : pd.Series(dtype=float) for meas_site in zones[ref_site]} for ref_site in zones.keys()}

test_times_lt_10pc_rsd = {ref_site : {meas_site : pd.Series() for meas_site in zones[ref_site]} for ref_site in zones.keys()}

drop_cols = ["timestamp", "co_corrected"]

for ref_site in zones.keys():
    for meas_site in zones[ref_site]:
        # Get the indices for train/test split based on the available timestamps for this measurement site
        train_indxs, test_indxs = train_test_split(range(len(meas_times_rsd_lt_10pc[meas_site])), random_state=0)
        
        # Select the timestamps for train/test
        train_times = meas_times_rsd_lt_10pc[meas_site].iloc[train_indxs]
        test_times = meas_times_rsd_lt_10pc[meas_site].iloc[test_indxs]
        test_times_lt_10pc_rsd[ref_site][meas_site] = test_times
        
        # Select the measurement data for train/test, dropping timestamp column
        X_train[ref_site][meas_site] = measurement_df[meas_site][measurement_df[meas_site]["timestamp"].isin(train_times)].set_index("timestamp")
        X_test_lt_10pc_rsd[ref_site][meas_site] = measurement_df[meas_site][measurement_df[meas_site]["timestamp"].isin(test_times)].set_index("timestamp")

        X_train[ref_site][meas_site] = X_train[ref_site][meas_site][["co", "temp", "rh"]]
        X_test_lt_10pc_rsd[ref_site][meas_site] = X_test_lt_10pc_rsd[ref_site][meas_site][["co", "temp", "rh"]]

        # Select the reference data for train/test, matching timestamps
        y_train[ref_site][meas_site] = reference_df[ref_site][reference_df[ref_site]["timestamp"].isin(train_times)].set_index("timestamp")["co"]
        y_test_lt_10pc_rsd[ref_site][meas_site] = reference_df[ref_site][reference_df[ref_site]["timestamp"].isin(test_times)].set_index("timestamp")["co"]
        
        dist_models[ref_site][meas_site].fit(X_train[ref_site][meas_site], y_train[ref_site][meas_site])
        y_test_pred_lt_10pc_rsd[ref_site][meas_site] = pd.Series(
            dist_models[ref_site][meas_site].predict(X_test_lt_10pc_rsd[ref_site][meas_site]),
            index=y_test_lt_10pc_rsd[ref_site][meas_site].index
        )
        # This is mapping to np.ndarrays when it should be mapping to a pd.Series

plot_zones(zones, dist_models, y_test_lt_10pc_rsd, y_test_pred_lt_10pc_rsd)

Test the trained models on timestamps outsite of the RSD(co)<0.10 set (i.e. RSD(co)>0.10).

In [ ]:
X_test_gt_10pc_rsd = {ref_site : {meas_site : pd.DataFrame() for meas_site in zones[ref_site]} for ref_site in zones.keys()}
y_test_gt_10pc_rsd = {ref_site : {meas_site : pd.Series(dtype=float) for meas_site in zones[ref_site]} for ref_site in zones.keys()}
y_test_pred_gt_10pc_rsd = {ref_site : {meas_site : pd.Series(dtype=float) for meas_site in zones[ref_site]} for ref_site in zones.keys()}

test_times_gt_10pc_rsd = {ref_site : {meas_site : pd.Series() for meas_site in zones[ref_site]} for ref_site in zones.keys()}

for ref_site in zones.keys():
    for meas_site in zones[ref_site]:
        
        test_times = meas_times_rsd_gt_10pc[meas_site] 
        test_times_gt_10pc_rsd[ref_site][meas_site] = test_times

        X_test_gt_10pc_rsd[ref_site][meas_site] = measurement_df[meas_site][measurement_df[meas_site]["timestamp"].isin(test_times)].set_index("timestamp")[['co', 'temp', 'rh']]
        y_test_gt_10pc_rsd[ref_site][meas_site] = reference_df[ref_site][reference_df[ref_site]["timestamp"].isin(test_times)].set_index("timestamp")["co"]
        y_test_pred_gt_10pc_rsd[ref_site][meas_site] = pd.Series(
            dist_models[ref_site][meas_site].predict(X_test_gt_10pc_rsd[ref_site][meas_site]), 
            index=y_test_gt_10pc_rsd[ref_site][meas_site].index
        )

plot_zones(zones, dist_models, y_test_gt_10pc_rsd, y_test_pred_gt_10pc_rsd)

Test the trained models on calibrated BEACO2N CO data.

In [ ]:
X_test_calib_co = {ref_site : {meas_site : pd.DataFrame() for meas_site in zones[ref_site]} for ref_site in zones.keys()}
y_test_calib_co = {ref_site : {meas_site : pd.Series(dtype=float) for meas_site in zones[ref_site]} for ref_site in zones.keys()}
y_pred_calib_co = {ref_site : {meas_site : pd.Series(dtype=float) for meas_site in zones[ref_site]} for ref_site in zones.keys()}

for ref_site in zones.keys():
    for meas_site in zones[ref_site]:
        X_test_calib_co[ref_site][meas_site] = pd.concat([
            X_test_lt_10pc_rsd[ref_site][meas_site], 
            X_test_gt_10pc_rsd[ref_site][meas_site]
        ], axis=0)

        y_pred_calib_co[ref_site][meas_site] = pd.concat([
            pd.Series(y_test_pred_lt_10pc_rsd[ref_site][meas_site]),
            pd.Series(y_test_pred_gt_10pc_rsd[ref_site][meas_site])
        ], axis=0)

        y_test_calib_co_times = pd.concat([
            test_times_lt_10pc_rsd[ref_site][meas_site],
            test_times_gt_10pc_rsd[ref_site][meas_site]
        ], axis=0)

        y_test_calib_co[ref_site][meas_site] = \
            measurement_df[meas_site][measurement_df[meas_site]["timestamp"].isin(y_test_calib_co_times)] \
                .set_index("timestamp", drop=True).reindex(y_test_calib_co_times).set_index(y_test_calib_co_times)["co_corrected"]
        

plot_zones(zones, dist_models, y_test_calib_co, y_pred_calib_co)


### Precision-Accuracy calibration
For every zone, we let the BEACO2N sensor that is colocated with reference to be M1 (Measurement_1). We then train a model for each non-colocated sensor Mn (model Pn) to predict M1's CO measurement, and we then train an accuracy model to predict R (Reference) CO data given M1 CO data. 

In [ ]:
# Find times within a zone when RSD(co)<0.10 for all sites
rsd_pa = {ref_site : [] for ref_site in zones.keys()}
ref_times_lt_rsd = {ref_site : pd.DataFrame() for ref_site in zones.keys()}
zone_times_lt_rsd = {ref_site : {meas_site : pd.Series() for meas_site in zones[ref_site]} for ref_site in zones.keys()}

for ref_site in zones.keys():
    rsd_pa[ref_site].append(reference_df[ref_site][["timestamp","co"]].rename(columns={"co":f"{ref_site}_ref"}))
    for meas_site in zones[ref_site]: 
        rsd_pa[ref_site].append(measurement_df[meas_site][["timestamp","co"]].rename(columns={"co":meas_site}))

    rsd_pa[ref_site][0] = reduce(lambda table, to_merge: pd.merge(table, to_merge, on="timestamp", how="inner"), rsd_pa[ref_site])
    rsd_pa[ref_site][0]["rsd"] = rsd_pa[ref_site][0].apply(rsd, axis=1)
    rsd_pa[ref_site][1] = rsd_pa[ref_site][0][rsd_pa[ref_site][0]["rsd"]>=1.20]["timestamp"]
    rsd_pa[ref_site][0] = rsd_pa[ref_site][0][rsd_pa[ref_site][0]["rsd"]<1.20]["timestamp"]
    if len(rsd_pa[ref_site])>2: rsd_pa[ref_site][2:].clear()
    
    for meas_site in zones[ref_site]: 
        zone_times_lt_rsd[ref_site][meas_site] = rsd_pa[ref_site][0]

        # Set zone_times_lt_rsd to be the set of timestamps for measurements in measurement_df[meas_site] that occured at times when the RSD(co) within the zone was less than 1.20
        zone_times_lt_rsd[ref_site][meas_site] = measurement_df[meas_site][measurement_df[meas_site]["timestamp"].isin(zone_times_lt_rsd[ref_site][meas_site])]["timestamp"]


In [ ]:
# Plotting function for accuracy models:
from sklearn.metrics import mean_absolute_error, r2_score

def plot_accuracy_models(zones, accuracy_model, y_test_a_lt, y_pred_a_lt):
    import matplotlib.pyplot as plt

    num_refsites = len(zones)
    fig, axes = plt.subplots(1, num_refsites, figsize=(4*num_refsites, 4), squeeze=False)
    statistics = []

    for col, ref_site in enumerate(zones.keys()):
        y_true = y_test_a_lt[ref_site]
        y_pred = y_pred_a_lt[ref_site]
        if len(y_true) == 0 or len(y_pred) == 0:
            axes[0][col].set_visible(False)
            continue
        residual = y_true - y_pred
        axes[0][col].scatter(y_true, residual, alpha=0.5)
        axes[0][col].axhline(0, color='red', linestyle='--')
        axes[0][col].set_title(f"{ref_site} Accuracy Model\nR^2={r2_score(y_true, y_pred):.3f}, MAE={mean_absolute_error(y_true, y_pred):.2f}")
        axes[0][col].set_xlabel("Reference CO (mV)")
        axes[0][col].set_ylabel("Residual (True - Pred) (mV)")

        model = accuracy_model[ref_site]
        statistics.append({
            "Reference": ref_site,
            "Determination R^2": r2_score(y_true, y_pred),
            "Correlation r": np.corrcoef(y_true, y_pred)[0, 1],
            "RMSE": sqrt(np.mean((y_true - y_pred) ** 2)),
            "MAE": mean_absolute_error(y_true, y_pred),
            "Intercept": model.intercept_,
            "co_coef": model.coef_[0],
            "temp_coef": model.coef_[1],
            "rh_coef": model.coef_[2],
            "n": len(y_true)
        })

    plt.tight_layout()
    plt.show()
    display(pd.DataFrame(statistics).round(4))


In [ ]:
precision_model = {ref_site : {meas_site : LinearRegression() for meas_site in zones[ref_site]} for ref_site in zones.keys()}
accuracy_model = {ref_site : LinearRegression() for ref_site in zones.keys()}

X_train_p = {ref_site : {meas_site : pd.DataFrame() for meas_site in zones[ref_site]} for ref_site in zones.keys()}
y_train_p = {ref_site : {meas_site : pd.Series() for meas_site in zones[ref_site]} for ref_site in zones.keys()}
X_train_a = {ref_site : pd.DataFrame() for ref_site in zones.keys()}
y_train_a = {ref_site : pd.Series() for ref_site in zones.keys()}

X_test_p_lt = {ref_site : {meas_site : pd.DataFrame() for meas_site in zones[ref_site]} for ref_site in zones.keys()}
y_test_p_lt = {ref_site : {meas_site : pd.Series() for meas_site in zones[ref_site]} for ref_site in zones.keys()}
y_pred_p_lt = {ref_site : {meas_site : pd.Series() for meas_site in zones[ref_site]} for ref_site in zones.keys()}

X_test_a_lt = {ref_site : pd.DataFrame() for ref_site in zones.keys()}
y_test_a_lt = {ref_site : pd.Series() for ref_site in zones.keys()}
y_pred_a_lt = {ref_site : pd.Series() for ref_site in zones.keys()}

for ref_site in zones.keys():
    for meas_site in zones[ref_site]:
        if meas_site == ref_site: continue

        train_indxs, test_indxs = train_test_split(range(len(zone_times_lt_rsd[ref_site][meas_site])), random_state=0, test_size=0.4)
        train_times = zone_times_lt_rsd[ref_site][meas_site].iloc[train_indxs].sort_index()
        test_times = zone_times_lt_rsd[ref_site][meas_site].iloc[test_indxs].sort_index()


        X_train_p[ref_site][meas_site] = measurement_df[meas_site][measurement_df[meas_site]["timestamp"].isin(train_times)][["co", "temp", "rh"]]
        X_test_p_lt[ref_site][meas_site] = measurement_df[meas_site][measurement_df[meas_site]["timestamp"].isin(test_times)][["co", "temp", "rh"]]

        y_train_p[ref_site][meas_site] = measurement_df[ref_site][measurement_df[ref_site]["timestamp"].isin(train_times)]["co"]
        y_test_p_lt[ref_site][meas_site] = measurement_df[ref_site][measurement_df[ref_site]["timestamp"].isin(test_times)]["co"]

        precision_model[ref_site][meas_site].fit(X_train_p[ref_site][meas_site], y_train_p[ref_site][meas_site])
        y_pred_p_lt[ref_site][meas_site] = precision_model[ref_site][meas_site].predict(X_test_p_lt[ref_site][meas_site]) # type: ignore

    X_train_a[ref_site] = measurement_df[ref_site][measurement_df[ref_site]["timestamp"].isin(train_times)][["co", "temp", "rh"]]
    X_test_a_lt[ref_site] = measurement_df[ref_site][measurement_df[ref_site]["timestamp"].isin(test_times)][["co", "temp", "rh"]]
    y_train_a[ref_site] = reference_df[ref_site][reference_df[ref_site]["timestamp"].isin(train_times)]["co"]
    y_test_a_lt[ref_site] = reference_df[ref_site][reference_df[ref_site]["timestamp"].isin(test_times)]["co"]
    accuracy_model[ref_site].fit(X_train_a[ref_site], y_train_a[ref_site])
    y_pred_a_lt[ref_site] = accuracy_model[ref_site].predict(X_test_a_lt[ref_site]) # type: ignore
    
plot_accuracy_models(zones, accuracy_model, y_test_a_lt, y_pred_a_lt)
plot_zones(zones, precision_model, y_test_p_lt, y_pred_p_lt)


Leave-one-out cross validation approach for p-a models: (since there is limited data for pha zone)